### Web data retrieval

In [ ]:
import requests

# !pip install requests

In [ ]:
#Let's download the data from inshorts website. In this case, news articles will be from 'technology' category
url = 'https://inshorts.com/en/read/technology'

In [ ]:
output = url.split('/')

In [ ]:
output

In [ ]:
output[-1]

In [ ]:
news_category = url.split('/')[-1]
news_category

Download HTML data

In [ ]:
#Download the data from Website
data = requests.get(url)

In [ ]:
print(data.content)

Data Cleaning - We can use Beautiful Soup package to clean Web data

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.content, 'html.parser')

In [ ]:
soup

Read all the articles. For each article, we will read:

1. Headline
2. Article body
3. Category

This is done by reading text between specific HTML tags. The tags depend on actual web page

In [ ]:
news_data = []

In [ ]:
all_news_headlines = soup.find_all('span', itemprop = ['headline'])

In [ ]:
all_news_headlines

In [ ]:
len(all_news_headlines)

In [ ]:
all_news_headlines[0].text

In [ ]:
for i in range(len(all_news_headlines)):
    print(i+1, all_news_headlines[i].text)

In [ ]:
all_news_body = soup.find_all('div', attrs={'itemprop': 'articleBody'})
all_news_body

In [ ]:
len(all_news_body)

In [ ]:
for i in range(len(all_news_body)):
    print(i+1, all_news_body[i].text)

In [ ]:
news_articles = [{'news_headline': headline.text,
                  'news_article': article.text,
                  'news_category': news_category}
                 for headline, article in zip(soup.find_all('span', itemprop = ['headline']),
                                              soup.find_all('div', attrs={'itemprop': 'articleBody'}))]

In [ ]:
lst1 = [1,2,3]
lst2 = [55,44,33]
outputlist = []

print(list(zip(lst1, lst2)))

for i,j in zip(lst1, lst2):
  outputlist.append(i+j)
print(outputlist)

In [ ]:
type(news_articles)

In [ ]:
news_articles

In [ ]:
news_articles[0]

In [ ]:
#Check news data
news_data.extend(news_articles)
news_data

Read the news data in a Dataframe

In [ ]:
import pandas as pd

In [ ]:
#Building dataframe
df = pd.DataFrame(news_data, columns=['news_headline', 'news_article', 'news_category'])

In [ ]:
df.head()

### Extract more Web data


Function to extract data from inshorts.com. The function will:

1. Take a URLs list as input
2. Get content for each URL
3. Extract news article headline, body and category

In [ ]:
urls_list = ['https://inshorts.com/en/read/technology',
             'https://inshorts.com/en/read/sports',
             'https://inshorts.com/en/read/world',
             'https://inshorts.com/en/read/businessm/en/read/wo']

In [ ]:
def datasetPrepare(urls_list):

    news_data = []
    for url in urls_list:
        news_category = url.split('/')[-1]
        data = requests.get(url)
        soup = BeautifulSoup(data.content, 'html.parser')
        news_articles = [{'news_headline': headline.text,
                  'news_article': article.text,
                  'news_category': news_category}
                 for headline, article in zip(soup.find_all('span', itemprop = ['headline']),
                                              soup.find_all('div', attrs={'itemprop': 'articleBody'}))]
        news_data.extend(news_articles)
    df =  pd.DataFrame(news_data)
    df = df[['news_headline', 'news_article', 'news_category']]
    return df

In [ ]:
#Build the dataframe
news_df = datasetPrepare(urls_list)

In [ ]:
news_df.sample(n=15)

In [ ]:
news_df.tail(2)

In [ ]:
news_df.to_csv("test.csv", index=False)

In [ ]:
#Articles count by category
news_df.news_category.value_counts()

### Text Preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

In [ ]:
import re
import unicodedata
from nltk.stem import WordNetLemmatizer

Remove HTML tags

In [ ]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

Remove accented characters

In [ ]:
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

Remove special characters

In [ ]:
def remove_special_characters(text, remove_digits=False):
    #Using regex
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

Lemmatization

In [ ]:
def lemmatize_text(text):

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

Stemming

In [ ]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

#### Building a text normalizer

In [ ]:
def normalize_corpus(corpus, html_stripping=True, accented_char_removal=True, text_lower_case=True,
                     text_lemmatization=True, special_char_removal=True,
                     stopword_removal=True, remove_digits=True):

    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # lowercase the text
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits
        if special_char_removal:
            # insert spaces between special characters to isolate them
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)

        normalized_corpus.append(doc)

    return normalized_corpus

### Pre-process and normalize news articles

In [ ]:
news_df['full_text'] = news_df["news_headline"].map(str)+ '. ' + news_df["news_article"]

In [ ]:
news_df.head()

In [ ]:
news_df['clean_text'] = normalize_corpus(news_df['full_text'])
norm_corpus = list(news_df['clean_text'])

In [ ]:
news_df.iloc[1][['full_text', 'clean_text']].to_dict()

### Save the news articles

In [ ]:
news_df.to_csv('news.csv', index=False, encoding='utf-8')

### Tokenization and Vectorization

TODO: Split data between training and text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Using it with regular text
vect = TfidfVectorizer()
vect.fit(news_df['clean_text'].tolist())
len(vect.get_feature_names_out())

In [ ]:
vect.vocabulary_